## **RAG의 핵심, 문서 검색기 Retriever**
### **사용자의 쿼리를 재해석하여 검색하다, MultiQueryRetriever**
**Chroma DB에 문서 벡터 저장**

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

#헌법 PDF 파일 로드
# PDF파일 불러올 객체 PyPDFLoader 선언 - macOS
loader = PyPDFLoader(
    r"/Users/youngho_moon/Library/CloudStorage/GoogleDrive-anskong@gmail.com/내 드라이브/LLM-RAG-LangChain/대한민국헌법(헌법)(제00010호)(19880225).pdf"
)
pages = loader.load_and_split()

#PDF 파일을 500자 청크로 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(pages)

#ChromaDB에 청크들을 벡터 임베딩으로 저장(OpenAI 임베딩 모델 활용)
db = Chroma.from_documents(docs, OpenAIEmbeddings(model = 'text-embedding-3-small'))


**질문을 여러 버전으로 재해석하여 Retriever에 활용**

In [ ]:
#```Chroma DB에 대한민국 헌법 PDF 임베딩 변환 및 저장하는 과정은 위 셀에 있습니다```
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI

#질문 문장 question으로 저장
question = "국회의원의 의무는 무엇이 있나요?"
#여러 버전의 질문으로 변환하는 역할을 맡을 LLM 선언
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125",
                 temperature = 0)
#MultiQueryRetriever에 벡터DB 기반 Retriever와 LLM 선언
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(), llm=llm
)

# 여러 버전의 문장 생성 결과를 확인하기 위한 로깅 과정
import logging
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

#여러 버전 질문 생성 결과와 유사 청크 검색 개수 출력
unique_docs = retriever_from_llm.invoke(input=question)
len(unique_docs)

In [ ]:
unique_docs